In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
id_train=pd.read_csv("/kaggle/input/titanic/train.csv")
id_test=pd.read_csv("/kaggle/input/titanic/test.csv")

In [1]:
id_train.head()

In [1]:
id_train.drop(["Name","PassengerId",'Cabin'],axis=1,inplace=True)
id_test.drop(["Name",'Cabin'],axis=1,inplace=True)

print(id_train.isnull().sum())
id_test.isnull().sum()

In [1]:
print(id_train.Sex.value_counts())
print(id_test.Sex.value_counts())

id_train['Sex']=id_train['Sex'].replace(['male','female'],[1,0])
id_test['Sex']=id_test['Sex'].replace(['male','female'],[1,0])

In [1]:
print(id_train.info())
print(id_test.info())

In [1]:
id_train.describe()

## includez only non object type column

In [1]:
id_train.describe(include=['O'])

In [1]:
id_train['Pclass'].value_counts()

## many passengers were in the 3rd class
## and less in 1st class

In [1]:
## lets see the survival rate with sex,SidSp, Parch and plcass with survived

In [1]:
id_train[['Sex','Survived']].groupby(['Sex']).mean().sort_values(by='Survived')

## 0 is female
## 1 is male

In [1]:
id_train[['SibSp','Survived']].groupby(['SibSp']).mean().sort_values(by='Survived',ascending=False)

## less the number of siblings onboard, more the probability of survival

In [1]:
id_train[['Parch','Survived']].groupby(['Parch']).mean().sort_values(by='Survived',ascending=False)

## more the children : less the survival probability

In [1]:
id_train[['Pclass','Survived']].groupby(['Pclass']).mean().sort_values(by='Survived',ascending=False)

## better the class : more the survival chances

#  **lets visualize the data**

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

v = sns.FacetGrid(id_train,col='Survived')
v.map(plt.hist,'Age',bins=15)

observation from above :
1. infants (age<=6) have high survival chances
2. Oldest passengers (age>75) survived.
3. age group between (15-30) survived less.


## lets do the survived vs age with respect to pclass

In [1]:
v = sns.FacetGrid(id_train,row='Survived',col='Pclass')
v.map(plt.hist,'Age',bins=15)

From above graph :
Pclass==1 : survived the most.

In [1]:

v = sns.FacetGrid(id_train,col='Embarked')
v.map(sns.pointplot,'Pclass','Survived','Sex',palette='deep')
v.add_legend()

## 0 is female
## 1 is male

## Observation

1. Overall, Females have high survival chances.
2. Males of pclass 2 in Embarked=="C" have higher survival chances than females in that category.

In [1]:
v=sns.FacetGrid(id_train,col='Embarked',row='Survived')
v.map(sns.barplot,'Sex','Fare')

## Observation
1. Higher fare paying people have more chances of survival

In [1]:
## from above observations, Age and Cabin have null value.
## lets fill those Na values by mean

id_train['Age'].fillna(id_train['Age'].mean(),inplace=True)
id_test['Age'].fillna(id_test['Age'].median(),inplace=True)

In [1]:
id_train.drop("Ticket",axis=1,inplace=True)
id_test.drop("Ticket",axis=1,inplace=True)

In [1]:
id_train.head()

In [1]:
## lets see the different age bands survival

id_train['ageband'] = pd.cut(id_train['Age'],5)
id_train[['ageband','Survived']].groupby('ageband').mean()

In [1]:
## lets convert the ages into groups for id_train
dataset=id_train
dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
dataset.loc[ dataset['Age'] > 64, 'Age']

id_train=dataset
id_train.head()

In [1]:
## lets convert the ages into groups for id_test 
dataset=id_test
dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
dataset.loc[ dataset['Age'] > 64, 'Age']

id_test=dataset
id_test.head()

In [1]:
id_train.drop('ageband',axis=1,inplace=True)

In [1]:
## lets create a new column of family size

id_train['family'] = id_train['Parch'] + id_train['SibSp'] + 1 

## for idtest
id_test['family'] = id_test['Parch'] + id_test['SibSp'] + 1 

id_train[['family','Survived']].groupby("family").mean()

In [1]:
id_train.head()

In [1]:
## if family is there : means ge is not alone


id_train['alone']=0

id_train.loc[id_train['family']==1,'alone']=1

id_train[['alone','Survived']].groupby("alone").mean()

In [1]:
## for idtest

## if family is there : means ge is not alone

id_test['alone']=0

id_test.loc[id_test['family']==1,'alone']=1



In [1]:
id_train = id_train.drop(['Parch', 'SibSp', 'family'], axis=1)
id_test = id_test.drop(['Parch', 'SibSp', 'family'], axis=1)

In [1]:
## embarked col has null values
## fill the na values with the most occurening embarking port

print(id_train['Embarked'].value_counts())
id_train['Embarked'] = id_train['Embarked'].fillna(('S'))

In [1]:
id_train[['Embarked','Survived']].groupby(["Embarked"]).mean()

In [1]:
## lets replace S:0; C:1; Q:2 in Embarked ports

id_train['Embarked'] = id_train['Embarked'].replace(['S','C','Q'],[0,1,2])
id_train['Embarked'].value_counts()

In [1]:
## lets replace S:0; C:1; Q:2 in Embarked ports in idtest

id_test['Embarked'] = id_test['Embarked'].replace(['S','C','Q'],[0,1,2])
id_test['Embarked'].value_counts()

In [1]:
id_train.head()

In [1]:
id_test.head()

In [1]:
## lets create the band price for fare

id_train['fareband'] = pd.qcut(id_train['Fare'],4)

id_train[['fareband','Survived']].groupby(["fareband"]).mean()

In [1]:
dataset=id_train

dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
dataset['Fare'] = dataset['Fare'].astype(int)

id_train = dataset

id_train.drop("fareband",axis=1,inplace=True)

id_train.head()

In [1]:

## theres null value in idtest['Fare']
id_test['Fare'].fillna(id_test['Fare'].mean(),inplace=True)

## fareband for idtest

dataset=id_test

dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
dataset['Fare'] = dataset['Fare'].astype(int)

id_test = dataset
id_test.head()

# Model Development

In [1]:
xtrain = id_train.drop('Survived',axis=1)
ytrain = id_train['Survived']
xtest = id_test.drop("PassengerId",axis=1).copy()

xtrain.shape, ytrain.shape,xtest.shape

## Logistic regression

In [1]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(xtrain,ytrain)
y_pred = logreg.predict(xtest)

log_score=logreg.score(xtrain,ytrain)
log_score

In [1]:
coeff_df = pd.DataFrame(id_train.columns.delete(0))
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

## SVC

In [1]:
from sklearn.svm import SVC, LinearSVC

svc=SVC()
svc.fit(xtrain,ytrain)
y_pred = svc.predict(xtest)

svc_score=svc.score(xtrain,ytrain)
svc_score

## KNN

In [1]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(xtrain,ytrain)
y_pred = knn.predict(xtest)

knn_score=knn.score(xtrain,ytrain)
knn_score

## Naive Bayes

In [1]:
from sklearn.naive_bayes import GaussianNB

gauss = GaussianNB()
gauss.fit(xtrain,ytrain)
y_pred = gauss.predict(xtest)

gauss_score=gauss.score(xtrain,ytrain)
gauss_score

## Linear SVC

In [1]:


svc=LinearSVC()
svc.fit(xtrain,ytrain)
y_pred=svc.predict(xtest)

lin_svc_score=svc.score(xtrain,ytrain)
lin_svc_score

## SGD

In [1]:
from sklearn.linear_model import SGDClassifier

sgd=SGDClassifier()
sgd.fit(xtrain,ytrain)
y_pred = sgd.predict(xtest)

sgd_score=sgd.score(xtrain,ytrain)
sgd_score

## Decision Tree

In [1]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(xtrain,ytrain)
y_pred = dt.predict(xtest)

dt_score=dt.score(xtrain,ytrain)
dt_score

## RandomForest

In [1]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(xtrain,ytrain)
y_pred_rf = rf.predict(xtest)

rf_score=rf.score(xtrain,ytrain)
rf_score

## CNN Model

In [1]:
xtrain.shape

In [1]:
from tensorflow import keras

model = keras.Sequential([
    
    ## reshaping the input entries
    keras.layers.Dense(50, input_shape=(6,), activation='relu'),  
    keras.layers.Dropout(0.50),    ## to avoid overfitting and underfiting

    ## creating the hidden layer
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dropout(0.70),    ##  to avoid overfitting and underfiting
    
    keras.layers.Dense(150,activation='relu'),
    keras.layers.Dropout(0.70),     ## to avoid overfitting and underfiting
 
    
    ## final neural layer
    keras.layers.Dense(1,activation='sigmoid')
    
])


model.compile(optimizer='adam',
             loss='binary_crossentropy',  ## since output in 0 or 1
             metrics=['accuracy'])

In [1]:
model.fit(xtrain,ytrain,epochs=100)

y_pred_cnn = model.predict(xtest)

cnn_score = model.evaluate(xtrain,ytrain)[1]
cnn_score

In [1]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree',"CNN"],
    'Score': [svc_score, knn_score, log_score, 
              rf_score, gauss_score, 
              sgd_score, lin_svc_score, dt_score,cnn_score]})
models.sort_values(by='Score', ascending=False)

Random Forest has the highest accuracy.
so we will submit the y_predicted value using random forest model

In [1]:
submission = pd.DataFrame({
        "PassengerId": id_test["PassengerId"],
        "Survived": y_pred_rf
    })

submission = submission.to_csv('submission.csv',index=False)




